In [1]:
from collections import defaultdict
import json
import omero.clients
from omero.gateway import BlitzGateway
from omero.rtypes import unwrap
from omero.sys import ParametersI
import os
import pandas as pd
import yaml

In [2]:
with open(os.path.expanduser('~/blitzgateway-credentials.yml')) as f:
    creds = yaml.safe_load(f)
conn = BlitzGateway(**creds, secure=True)
c = conn.connect()
assert c

In [3]:
projects = dict((p.name.split('/')[1], p) for p in conn.getObjects('Project') if p.name.startswith('idr0071'))
datasets = dict((pname, dict((d.name, d) for d in p.listChildren())) for (pname, p) in projects.items())

In [4]:
images = defaultdict(dict)
for p in projects.values():
    for d in p.listChildren():
        print(f'{p.name} / {d.name}')
        params = ParametersI()
        params.addId(d.id)
        rs = conn.getQueryService().projection(
            'SELECT link.child.name, link.child.id FROM DatasetImageLink link WHERE link.parent.id=:id',
            params)
        images[p.name][d.name] = dict(unwrap(rs))
conn.close()

idr0071-feldman-crisprko/experimentE / aligned A11 process
idr0071-feldman-crisprko/experimentE / aligned B10 process
idr0071-feldman-crisprko/experimentE / aligned B11 process
idr0071-feldman-crisprko/experimentE / aligned B12 process
idr0071-feldman-crisprko/experimentE / aligned C10 process
idr0071-feldman-crisprko/experimentE / aligned C11 process
idr0071-feldman-crisprko/experimentE / aligned C12 process
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs A11 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs B10 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs B11 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs B12 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs C10 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs C11 raw
idr0071-feldman-crisprko/experimentE / c10-SBS-10 sbs C12 raw
idr0071-feldman-crisprko/experimentE / c11-SBS-11 sbs A11 raw
idr0071-feldman-crisprko/experimentE / c11-SBS-11 sbs B10 raw
idr0071-feldman-crisprko/expe

In [5]:
with open('imagelist.json', 'w') as f:
    json.dump(images, f)